# Deep Convolutional Neural Network in TensorFlow

In this notebook, we build a deep, convolutional, MNIST-classifying network inspired by [LeNet-5](http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jonkrohn/DLTFpT/blob/master/notebooks/lenet_in_tensorflow.ipynb)

#### Load dependencies

In [1]:
import tensorflow
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
# conv2D helps to create convolution layer
# maxPooling2D to apply max pooling
# Note: both dependecies have suffix 2D because they operate on 2D data, 
# there are other flavors of both libraries for 1D and 3D 
# 1D used in stock price prediction, 3D used in complex medical image analysis or video analysis.
from tensorflow.keras.layers import Flatten, Conv2D, MaxPooling2D # new!

#### Load data

In [2]:
(X_train, y_train), (X_valid, y_valid) = mnist.load_data()

#### Preprocess data

In [3]:
# if you notice second parameter is not 784 = 28 * 28 unlike in normal ANN. shallow_net_in_tensorflow.ipynb
# We want to pass 2D input to the convolution layer, so we are passing as 28, 28
# last parameter 1 indicates that there is only 1 layer - because our data is white and black
# if our input data was colored image, then it would have RGB layers, we should pass 3 in that case.

X_train = X_train.reshape(60000, 28, 28, 1).astype('float32')
X_valid = X_valid.reshape(10000, 28, 28, 1).astype('float32')

In [4]:
X_train /= 255
X_valid /= 255

In [5]:
n_classes = 10
y_train = to_categorical(y_train, n_classes)
y_valid = to_categorical(y_valid, n_classes)

#### Design neural network architecture

In [6]:
model = Sequential()

# First Hidden Layer:
# We are adding convolution layer (which contains filters), not dense layer.
# we have to specify filter size/kernel size = 3 * 3
# once we perform wx+ b = z, we have to apply activation on that z, that activtion function is reLu
# input shape is again 28 * 28 and 1 layer.
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))

# Second Hidden Layer:
# convolution layer with filter size 3 * 3
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))

# we are adding maxPooling layer,
# (2,2) means it will form subset of (2,2) and finds maximum element from that sub matrix.
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
# flatten will convert higher dimensional data to 1D array.
# this is required because next layer is dense layer, dense layer requires 1D input only.
model.add(Flatten())

model.add(Dense(128, activation='relu'))
# we have to apply more dropout in hidden layer close to output layer, than it's preceeding layer.
model.add(Dropout(0.5))

model.add(Dense(n_classes, activation='softmax'))

/Users/sshivagangeprakash/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 24, 24, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,179,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,199,882 (4.58 MB)

 Trainable params: 1,199,882 (4.58 MB)

 Non-trainable params: 0 (0.00 B)

#### Configure model

In [8]:
model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])

#### Train!

In [9]:
model.fit(X_train, y_train, batch_size=128, epochs=10, verbose=1, validation_data=(X_valid, y_valid))

Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 26s 54ms/step - accuracy: 0.8478 - loss: 0.4792 - val_accuracy: 0.9844 - val_loss: 0.0521
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 25s 53ms/step - accuracy: 0.9720 - loss: 0.0909 - val_accuracy: 0.9890 - val_loss: 0.0371
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 24s 51ms/step - accuracy: 0.9801 - loss: 0.0665 - val_accuracy: 0.9884 - val_loss: 0.0357
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 25s 53ms/step - accuracy: 0.9828 - loss: 0.0555 - val_accuracy: 0.9906 - val_loss: 0.0300
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 24s 52ms/step - accuracy: 0.9863 - loss: 0.0441 - val_accuracy: 0.9909 - val_loss: 0.0298
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 24s 52ms/step - accuracy: 0.9876 - loss: 0.0386 - val_accuracy: 0.9887 - val_loss: 0.0324
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 27s 57ms/step - accuracy: 0.9889 - loss: 0.0327 - val_accuracy: 0.9902 - val_loss: 0.0310
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 27s 58ms/step - accuracy: 0.9899 - loss: 0.0316 - 